In [104]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import h5py
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
import time

In [53]:
base_dir="https://magic.wizards.com/"
base_url="https://magic.wizards.com/en/content/deck-lists-magic-online-products-game-info"

In [106]:
# create a new Firefox session
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(base_url)
driver.maximize_window()

In [107]:
driver.execute_script("window.scrollTo(0, 200)") 
Filterby=driver.find_element_by_id('event_filter')
Filterby.click()
time.sleep(1)
Modern_button=driver.find_element_by_xpath("//li[contains(text(), 'Modern')]")
Modern_button.click()
time.sleep(1)
Search_button=driver.find_element_by_id('custom-search-submit')
Search_button.click()

# select by visible text
# select.select_by_visible_text('Modern')

In [ ]:
#After opening the url above, Selenium clicks the specific agency link


SCROLL_PAUSE_TIME = 1

timeout = 60



while True:

    # Get scroll height
    ### This is the difference. Moving this *inside* the loop
    ### means that it checks if scrollTo is still scrolling 
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    
    # touch da fishy
    da_fishy = driver.find_element_by_class_name('see-more-article-listing-section') #See more
    da_fishy.click() #click see more

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    i=0
    while (new_height == last_height) and (i<timeout) :

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        time.sleep(1)
        i+=1
    if i==timeout:
        break
#         try:
#             WebDriverWait(driver, timeout).until(new_height != last_height)
#         except TimeoutException: 
#             print ("Timed out waiting for page to load")


In [93]:
last_height

2294

In [51]:

r =  requests.get(base_url)
c = r.content
soup=BeautifulSoup(c,"html.parser")

In [ ]:
# Opens all links and gets all data
links=[]
for link in soup.find("div",{"class":"articles-listing"}).find_all('a'):
    a=link.get('href')
    links.append(a)

In [ ]:

deck_name_ls=[]
card_type_ls=[]
card_type_ls_string=[]
for page in links:
    page_url=base_dir+page
    r=requests.get(page_url)
    c=r.content
    page_soup=BeautifulSoup(c,"html.parser")
#     page_list.append(page_soup)
    for deckname in page_soup.find_all("div",{"class":"deck-group"}):
        deck_name=Data_set.require_group(deckname.find("span",{"class":"deck-meta"}).h4.string)
        deck_name_ls.append(deck_name.name)
        card_type=[]
        card_count=[]
        card_name=[]
        card_link=[]
        sideb_type=[]
        sideb_count=[]
        sideb_name=[]
        sideb_link=[]
        for card in deckname.find("div", {"class":"sorted-by-overview-container"}).find_all("div",{"class":"element"}):
            card_type.append(np.chararray.encode(card.h5.text,'utf8'))
            card_type_ls.append(card.h5.text)
            card_type_ls_string.append(card.h5.text)
            card_count.append(card.find("span",{"class":"card-count"}).text)
            card_name.append(card.find("span",{"class":"card-name"}).text)
            card_link.append(card.find("span",{"class":"card-name"}).a.get('href'))
#             sideb_type.append(np.chararray.encode(card.find("div", {"class":"sorted-by-sideboard-container"}).h5.text, 'utf8'))
        for sideb in deckname.find("div", {"class":"sorted-by-sideboard-container"}).find_all("span",{"class":"row"}):
            sideb_count.append(np.chararray.encode(sideb.find("span",{"class":"card-count"}).text, 'utf8'))
            sideb_name.append(np.chararray.encode(sideb.find("span",{"class":"card-name"}).text, 'utf8'))
            sideb_link.append(np.chararray.encode(sideb.find("span",{"class":"card-name"}).a.get('href'), 'utf8'))
                
        deck=deck_name.require_group('Deck')
        ctype=deck.require_group('Card Types')
        ccount=deck.require_group('Card Counts')
        cname=deck.require_group('Card Names')
        clink=deck.require_group('Card Links')
        sideb=deck_name.require_group('Sideboard')
        sideb_ctype=sideb.require_group('Sideboard Card Types')
        sideb_ccount=sideb.require_group('Sideboard Card Counts')
        sideb_cname=sideb.require_group('Sideboard Card Names')
        sideb_clink=sideb.require_group('Sideboard Card Links')
        
        ctype.create_dataset('Card Type',(len(card_type),1), str,card_type)
        ccount.create_dataset('Card Count',(len(card_count),1),str,card_count)
        cname.create_dataset('Card Name',(len(card_name),1),str,card_name)
        clink.create_dataset('Card Link',(len(card_link),1),str,card_link)

#         sideb_ctype.create_dataset('Card Type',(len(sideb_type),1),'S13',card_type)
#         sideb_ccount.create_dataset('Card Count',(len(sideb_count),1),'S13',card_count)
#         sideb_cname.create_dataset('Card Name',(len(sideb_name),1),'S13',card_name)
#         sideb_clink.create_dataset('Card Link',(len(sideb_link),1),'S13',card_link)
                                 
                                 
#         for a in range(len(temp_deck)):
#             decks_list.append(temp_deck[a])
#         for deck in decks.find("div",{"class":"deck-list-text"}):
#             CreatureandNumber.append(deck.find('h5'))
#         for deck_attr in page_soup.find("div",{"class":"deck-list-text"}):
#             for card_attr in page_soup.find("div", {"class":"sorted-by-overview-container"}):

In [ ]:
Data_set.close()

In [ ]:
decks=[]
for i in range(len(pages)):
    decks.append(pages[i].find("div",{"class":"decklists"}))

    

In [ ]:
deck=[]
for j in range(len(decks)):
        deck.append(decks[j].find("div",{"class":"deck-list-text"}))

In [ ]:
deck[0].find_all('h5')[0].text

In [ ]:
card_type_ls_string

In [ ]:
(len(card_type),1)

In [ ]:
encode(card_link,"ascii","ignore")

In [ ]:
card_type_ls_string[0]

In [ ]:
test=h5py.File('test.hdf5')

In [ ]:
testsub=test.create_group('test2')

In [ ]:
deck_name.